<a href="https://colab.research.google.com/github/yt1970/ai_engineering_03_homework/blob/main/PDFtoChank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 参照ドキュメントであるPDFをチャンクに分けて、JSON化するコード



本コードは、松尾研、AIエンジニアリング講座2025Day３の任意宿題用に作成しました。

○　PDFをテキスト化するライブラリー

　　PyPDFは演習ファイルのReadmeを参照しました。→
[PyPDF](https://pypi.org/project/PyPDF2/)


○　参照するドキュメント

　　[防衛力抜本的強化の
進捗と予算](https://www.mod.go.jp/j/budget/yosan_gaiyo/fy2025/yosan_20250402.pdf)（令和７年４月２日）


   参照するPDFは防衛省HPから令和７年度予算の概要の予算パンフレットを使用しました。同パンフレットは令和７年４月２日HP上に公開されていますので、RAGを使わなければLLMでは正確な回答ができないものと思われます。








###（下準備）PyPDFを使ってPDFをテキスト化します。

In [1]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
datafolder = "/content/drive/MyDrive/Colab Notebooks/AIengineering2025_colab/DAY3/data/"
pdfname = "yosan_20250402.pdf"
pdfpath = datafolder + pdfname

とりあえず、ページごとに区切り（--- page break ---）を入れて読み取ります。

In [4]:
from PyPDF2 import PdfReader


reader = PdfReader(pdfpath)
number_of_pages = len(reader.pages)

# 1ページ目全頁を取得間に \n--- page break ---\n
text = ""
for page in reader.pages[0:]:
    text += page.extract_text() #+ "\n--- page break ---\n"


読み取った結果を確認します。

In [ ]:
print(text)

防
衛
省
Ministry of  Defense
防衛力抜本的強化の
進捗と予算
令和７年度予算の概要
--- page break ---
目次
Ⅰ
防衛力整備計画の進捗状況
Ⅱ
令和７年度予算
Ⅲ
主要事項
１スタンド・オフ防衛能力
２統合防空ミサイル防衛能力
３無人アセット防衛能力
４領域横断作戦能力
５指揮統制・情報関連機能
６機動展開能力・国民保護
７持続性・強靱性
Ⅳ
共通基盤
１防衛生産基盤の強化
２研究開発
３早期装備化のための取組
４ＡＩ活用の推進に係る施策
５防衛力を支える要素
６日米同盟強化及び地域社会との調和に係る施策等
７安全保障協力の強化
８気候変動への取組
９最適化への取組
１０ 自衛隊の組織編成
１１ 自衛官の定員
１２ 事務官等の増員
１３ 情報保全の強化
１４ 監察体制の強化
１５ 税制改正1
--- page break ---
7つの重視分野 抜本的強化の進捗状況 (一例)
スタンド・オフ
防衛能力〇トマホークの 取得１年前倒し
(2026 年度→2025年度)
〇トマホーク運用のための海上自衛官等に
対する 要員養成教育 を開始
(2024 年3月～10月)
〇島嶼防衛用高速滑空弾の 事前発射試験 実施
(2024 年4月)
〇１２ 式地対艦誘導弾能力向上型 (地発型 )の
配備１年前倒し (2026 年度→2025年度)
〇12式地対艦誘導弾能力向上型の
発射試験 実施(2024 年10～11月)
統合防空ミサイル
防衛能力○イージス・システム搭載艦 2隻の建造契約 を締結 (2024 年8・9月)
○ＧＰＩの日米共同開発に係る 開発コンセプト を決定 (2024 年9月)
※GPI: Glide Phase Interceptor
(滑空段階迎撃用誘導弾 )
無人アセット
防衛能力○2023年度中に 合計１０機種の各種ＵＡＶの運用実証 を実施
○各種無人アセットに関する 研究開発 を推進
○滞空型ＵＡＶの機種として 「ＭＱ ―９Ｂ(シーガーディアン )」
を選定 (2024 年11月)
Ⅰ
防衛力整備計画の進捗状況
○我が国の防衛上必要な機能・能力として、次の７つの分野を重視して、
防衛力の抜本的強化を推進。
○事業の進捗管理を徹底し、概ね計画どおり進捗。
○令和７年度予算においても、計画期間内の防衛力抜本的強化実現

In [5]:
# # 保存するテキスト
# text = "これはサンプルのテキストです。"

# ファイルに保存
with open(datafolder + "row_page_sep.txt", "w", encoding="utf-8") as f:
    f.write(text)

### **Step1** テキスト整形なしでページごとにチャンクにする。


読み取ったテキストは、変なところに改行があったり、スペースがあったりしますが整形の効果を見るためにそのままチャンクにします。チャンクの単位はページごとにします。

作成したJSONファイル名は

**step1_raw_pages.json**

とします。

In [ ]:
import json
from PyPDF2 import PdfReader

class PDFChunkerPageRaw:
    def __init__(self, pdf_path):
        self.reader = PdfReader(pdf_path)
        self.pages_text = [page.extract_text() or "" for page in self.reader.pages]

    def chunk_by_page(self):
        chunks = []
        for i, text in enumerate(self.pages_text):
            title = f"page_{i+1}"
            chunks.append((title, text.strip()))
        return chunks

    def save_to_json(self, chunks, out_path):
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(
                [{"title": title, "content": content} for title, content in chunks],
                f, ensure_ascii=False, indent=2
            )

# 実行部分（例）
# if __name__ == "__main__":
#     pdf_path = "your_file.pdf"  # ← 対象PDFファイルのパス
#     out_path = "step1_raw_pages.json"

#     chunker = PDFChunkerPageRaw(pdf_path)
#     chunks = chunker.chunk_by_page()
#     chunker.save_to_json(chunks, out_path)

In [ ]:
pdf_path = pdfpath
out_path = datafolder + "pattern1_raw_pages.json"

chunker = PDFChunkerPageRaw(pdf_path)
chunks = chunker.chunk_by_page()
chunker.save_to_json(chunks, out_path)

できたJSONファイルを確認します。

In [ ]:
import json

with open(datafolder + "pattern1_raw_pages.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 最初の3件だけ表示
for i, item in enumerate(data[:3]):
    print(f"--- Chunk {i+1} ---")
    print("Title:", item["title"])
    print("Content:", item["content"][:100], "...\n")  # 長すぎる場合は先頭だけ

--- Chunk 1 ---
Title: page_1
Content: 防
衛
省
Ministry of  Defense
防衛力抜本的強化の
進捗と予算
令和７年度予算の概要 ...

--- Chunk 2 ---
Title: page_2
Content: 目次
Ⅰ
防衛力整備計画の進捗状況
Ⅱ
令和７年度予算
Ⅲ
主要事項
１スタンド・オフ防衛能力
２統合防空ミサイル防衛能力
３無人アセット防衛能力
４領域横断作戦能力
５指揮統制・情報関連機能
６機動 ...

--- Chunk 3 ---
Title: page_3
Content: 7つの重視分野 抜本的強化の進捗状況 (一例)
スタンド・オフ
防衛能力〇トマホークの 取得１年前倒し
(2026 年度→2025年度)
〇トマホーク運用のための海上自衛官等に
対する 要員養成教育  ...



### **Step2** テキストを整形して、ページごとにチャンクにする。


次にテキストを整形します。具体的には以下の通りです。


*   無駄な空白を削除する。（単語間に紛れたスペース、文頭や行頭のスペース）
*   有用な改行を残しつつ、無駄な改行を削除する。（行頭文字の判定。行末文字の判定。文字数などで判定）

* 抜けている改行を補う。（行頭文字の判定。行末文字の判定。文字数などで判定）



In [ ]:
import re

def normalize_spaces(text): # 無駄な空白を削除する。
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        line = line.strip()

        # 全角文字同士の間の空白を削除
        line = re.sub(r'(?<=[^\x00-\x7F])\s+(?=[^\x00-\x7F])', '', line)

        # 全角 → 半角、半角 → 全角 の間の空白も削除
        line = re.sub(r'(?<=[^\x00-\x7F])\s+(?=[\x00-\x7F])', '', line)  # 全→半
        line = re.sub(r'(?<=[\x00-\x7F])\s+(?=[^\x00-\x7F])', '', line)  # 半→全

        cleaned_lines.append(line)

    return "\n".join(cleaned_lines)

In [ ]:
def remove_linebreaks(text):
    # 「。」以外の改行を削除し、文をつなげる
    lines = text.splitlines()
    buffer = ""
    for line in lines:
        line = line.strip()
        if line:
            if line.endswith("。"):
                buffer += line + "\n"
            else:
                buffer += line  # 改行を除去して連結
    return buffer.strip()

In [ ]:
import re

def add_missing_linebreaks(text):
    pattern = re.compile(
        r'(?<!\n)'  # 直前が改行でない
        r'('
        r'(?:[○◎※●■◆◇＊①-⑩⑴⑵⑶⑷〇])'  # 全角記号（「・」除外、〇追加）
        r'|(?:\(?\d+[\.\)])'  # (1), 1. など
        r'|(?:第?[０-９\d]+[章部回])'  # 第１章、第２部
        r'|(?:[ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩⅪⅫⅰⅱⅲⅳⅴⅵⅶⅷⅸⅹ])'  # 全角ローマ数字
        r'|(?:\b[IVXLCDM]{1,4}\b)'  # 半角ローマ（大文字）
        r'|(?:\b[ivxlcdm]{1,4}\b)'  # 半角ローマ（小文字）
        r'|(?:(?<![０-９])([０-９]{1,3})(?!年|個|つ|日|円|％|点|名|機|隻|両|式))'  # 全角数字の後に特定の単位がつかない
        # r'|(?:(?<![０-９]))(?![Ａ-Ｚａ-ｚ０-９])'  # 全角英数字（除外）
        r'|(?<![０-９])([○◎※●■◆◇＊①-⑩⑴⑵⑶⑷〇])'  # 全角記号（除外）
        r')'
    )

    def insert_linebreak(match):
        return '\n' + match.group(0)

    return pattern.sub(insert_linebreak, text)

In [ ]:
def full_cleaning_pipeline(raw_text):
    # ステップ1: 不要な空白削除（改行は壊さない）
    text = normalize_spaces(raw_text)

    # ステップ2: 改行整形（意味ある改行を保持、それ以外は連結）
    text = remove_linebreaks(text)

    # ステップ3: 抜けている改行（記号などで段落頭と判断される行）を補う
    text = add_missing_linebreaks(text)

    return text

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader(pdfpath)
pages = [page.extract_text() or "" for page in reader.pages]

cleaned_pages = [full_cleaning_pipeline(page) for page in pages]

# 確認
for i, page in enumerate(cleaned_pages[:3]):
    print(f"--- Page {i+1} ---\n{page}\n")

--- Page 1 ---
防衛省Ministry of  Defense防衛力抜本的強化の進捗と予算令和７年度予算の概要

--- Page 2 ---
目次
Ⅰ防衛力整備計画の進捗状況
Ⅱ令和７年度予算
Ⅲ主要事項
１スタンド・オフ防衛能力
２統合防空ミサイル防衛能力
３無人アセット防衛能力
４領域横断作戦能力
５指揮統制・情報関連機能６機動展開能力・国民保護
７持続性・強靱性
Ⅳ共通基盤
１防衛生産基盤の強化
２研究開発
３早期装備化のための取組
４ＡＩ活用の推進に係る施策
５防衛力を支える要素６日米同盟強化及び地域社会との調和に係る施策等
７安全保障協力の強化
８気候変動への取組
９最適化への取組
１０自衛隊の組織編成
１１自衛官の定員
１２事務官等の増員
１３情報保全の強化
１４監察体制の強化
１５税制改正1

--- Page 3 ---
7つの重視分野抜本的強化の進捗状況(一例)スタンド・オフ防衛能力
〇トマホークの取得１年前倒し(2026年度→2025年度)
〇トマホーク運用のための海上自衛官等に対する要員養成教育を開始(2024年3月～10月)
〇島嶼防衛用高速滑空弾の事前発射試験実施(2024年4月)
〇
１２式地対艦誘導弾能力向上型(地発型)の配備１年前倒し(2026年度→2025年度)
〇12式地対艦誘導弾能力向上型の発射試験実施(2024年10～11月)統合防空ミサイル防衛能力
○イージス・システム搭載艦2隻の建造契約を締結(2024年8・9月)
○ＧＰＩの日米共同開発に係る開発コンセプトを決定(2024年9月)
※GPI: Glide Phase Interceptor(滑空段階迎撃用誘導弾)無人アセット防衛能力
○2023年度中に合計
１０機種の各種ＵＡＶの運用実証を実施
○各種無人アセットに関する研究開発を推進
○滞空型ＵＡＶの機種として「ＭＱ―
９Ｂ(シーガーディアン)」を選定(2024年11月)
Ⅰ防衛力整備計画の進捗状況
○我が国の防衛上必要な機能・能力として、次の７つの分野を重視して、防衛力の抜本的強化を推進。
○事業の進捗管理を徹底し、概ね計画どおり進捗。
○令和７年度予算においても、計画期間内の防衛力抜本的強化実現のため、令和７年度中に着手すべき事業を計上。
島嶼防衛用高速滑空弾トマホーク(イメージ)イージス・システム搭

完璧ではありませんが、いい感じに整形できました。

ページごとにチャンク化し、JSONファイルで保存します。

ファイル名は

**pattern2_cleand_pages.json**

です。

In [ ]:
import json
from PyPDF2 import PdfReader

class PDFChunkerPageclean:
    def __init__(self, pdf_path):
        self.reader = PdfReader(pdf_path)
        self.raw_pages = [page.extract_text() or "" for page in self.reader.pages]

    def clean_pages(self):
        return [full_cleaning_pipeline(page) for page in self.raw_pages]

    def chunk_by_page(self, cleaned_pages):
        return [(f"page_{i+1}", text.strip()) for i, text in enumerate(cleaned_pages)]

    def save_to_json(self, chunks, out_path):
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(
                [{"title": title, "content": content} for title, content in chunks],
                f, ensure_ascii=False, indent=2
            )

In [ ]:
pdf_path = pdfpath
out_path = datafolder + "pattern2_cleand_pages.json"

processor = PDFChunkerPageclean(pdf_path)
cleaned = processor.clean_pages()
cleandchunks = processor.chunk_by_page(cleaned)
processor.save_to_json(cleandchunks, out_path)

Jsonファイルを確認します。

In [ ]:
import json

with open(out_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 最初の3件だけ表示
for i, item in enumerate(data[:5]):
    print(f"--- Chunk {i+1} ---")
    print("Title:", item["title"])
    print("Content:", item["content"][:2000], "...\n")  # 長すぎる場合は先頭だけ

--- Chunk 1 ---
Title: page_1
Content: 防衛省Ministry of  Defense防衛力抜本的強化の進捗と予算令和７年度予算の概要 ...

--- Chunk 2 ---
Title: page_2
Content: 目次
Ⅰ防衛力整備計画の進捗状況
Ⅱ令和７年度予算
Ⅲ主要事項
１スタンド・オフ防衛能力
２統合防空ミサイル防衛能力
３無人アセット防衛能力
４領域横断作戦能力
５指揮統制・情報関連機能６機動展開能力・国民保護
７持続性・強靱性
Ⅳ共通基盤
１防衛生産基盤の強化
２研究開発
３早期装備化のための取組
４ＡＩ活用の推進に係る施策
５防衛力を支える要素６日米同盟強化及び地域社会との調和に係る施策等
７安全保障協力の強化
８気候変動への取組
９最適化への取組
１０自衛隊の組織編成
１１自衛官の定員
１２事務官等の増員
１３情報保全の強化
１４監察体制の強化
１５税制改正1 ...

--- Chunk 3 ---
Title: page_3
Content: 7つの重視分野抜本的強化の進捗状況(一例)スタンド・オフ防衛能力
〇トマホークの取得１年前倒し(2026年度→2025年度)
〇トマホーク運用のための海上自衛官等に対する要員養成教育を開始(2024年3月～10月)
〇島嶼防衛用高速滑空弾の事前発射試験実施(2024年4月)
〇
１２式地対艦誘導弾能力向上型(地発型)の配備１年前倒し(2026年度→2025年度)
〇12式地対艦誘導弾能力向上型の発射試験実施(2024年10～11月)統合防空ミサイル防衛能力
○イージス・システム搭載艦2隻の建造契約を締結(2024年8・9月)
○ＧＰＩの日米共同開発に係る開発コンセプトを決定(2024年9月)
※GPI: Glide Phase Interceptor(滑空段階迎撃用誘導弾)無人アセット防衛能力
○2023年度中に合計
１０機種の各種ＵＡＶの運用実証を実施
○各種無人アセットに関する研究開発を推進
○滞空型ＵＡＶの機種として「ＭＱ―
９Ｂ(シーガーディアン)」を選定(2024年11月)
Ⅰ防衛力整備計画の進捗状況
○我が国の防衛上必要な機能・能力として、次の７つの分野を重視して、防衛力の抜本的強化を推進。
○事業の進捗管理を徹底し、概ね計画どおり進捗。
○令和７

## 質問及び回答のJSONファイルへの格納

In [ ]:
import json

data_to_save = [
  {
    "質問": "令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。",
    "模範解答": "令和7年度の防衛費の予算総額は**7兆9,170億円**です。\n\nその主要な内訳は以下の通りです。\n\n* **装備品費等:** 2兆8,354億円（新たな装備品の取得、既存装備品の改修などに係る経費）\n* **隊員人件・糧食費:** 2兆2,682億円（自衛隊員の給与、食料、被服などに係る経費）\n* **施設整備・維持費等:** 7,305億円（基地・駐屯地の整備、維持管理、借料などに係る経費）\n* **教育訓練等:** 6,845億円（隊員の教育訓練、演習などに係る経費）\n* **研究開発費:** 3,812億円（将来の装備品や技術に関する研究開発に係る経費）\n* **その他:** 1兆2,172億円（上記の主要な経費に含まれないその他の経費）",
    "出典": "1ページ「令和７年度予算の概要」内「１．歳出」の表"
  },
  {
    "質問": "防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。",
    "模範解答": "防衛力整備計画で重視されている7つの分野は以下の通りです。\n\n1.  **スタンド・オフ防衛能力の強化:**\n    * **具体的な取り組み例:** 島嶼防衛用高速滑空弾の取得、長射程巡航ミサイルの開発・取得\n2.  **統合防空ミサイル防衛能力の強化:**\n    * **具体的な取り組み例:** イージス・システム搭載艦の建造、地対空誘導弾パトリオット（PAC-3）の能力向上型への換装\n3.  **無人アセット防衛能力の強化:**\n    * **具体的な取り組み例:** 各種無人航空機（UAV）の導入、無人水中航走体（UUV）の研究開発\n4.  **宇宙・サイバー・電磁波領域における能力の強化:**\n    * **具体的な取り組み例:** 衛星コンステレーションの構築、サイバー攻撃対処能力の向上、電磁波領域における優勢確保のための装備開発\n5.  **情報機能の強化:**\n    * **具体的な取り組み例:** 各種センサーの情報収集能力向上、収集した情報の分析・共有基盤の強化\n6.  **輸送能力の強化:**\n    * **具体的な取り組み例:** 大型輸送機の取得、輸送艦の建造\n7.  **持続性・強靭性の強化:**\n    * **具体的な取り組み例:** 弾薬・燃料等の備蓄拡充、重要施設の強靭化",
    "出典": "2ページ「２．主要経費」内「（１）重点的な整備・強化を行う主要分野」の箇条書き"
  },
  {
    "質問": "過去5年間の日本の防衛費の推移を説明してください。特に、GDP比での変化に着目してください。",
    "模範解答": "資料には令和7年度の予算概要のみが記載されており、過去5年間の防衛費の推移に関する具体的な数値データは含まれていません。そのため、この資料のみで過去5年間の防衛費の推移とGDP比での変化を説明することはできません。",
    "出典": "資料全体（過去のデータに関する記載なし）"
  },
  {
    "質問": "令和7年度の防衛予算における研究開発費の割合はどのくらいですか？ また、研究開発費は具体的にどのような分野に投資されていますか？",
    "模範解答": "令和7年度の防衛予算総額は7兆9,170億円であり、研究開発費は3,812億円です。したがって、研究開発費の割合は、\n\n$$\\frac{3,812 \\text{億円}}{79,170 \\text{億円}} \\times 100 \\approx 4.81 \%$$\n\n研究開発費は、将来の装備品や技術に関する広範な分野に投資されています。具体的には、以下の分野などが挙げられています。\n\n* **スタンド・オフ防衛能力:** 島嶼防衛用高速滑空弾、長射程巡航ミサイル関連技術\n* **統合防空ミサイル防衛能力:** 新型レーダー、迎撃ミサイル関連技術\n* **無人アセット防衛能力:** 無人航空機（UAV）、無人水中航走体（UUV）関連技術\n* **宇宙・サイバー・電磁波領域における能力:** 衛星システム、サイバーセキュリティ技術、電磁波兵器関連技術\n* **その他基盤的な研究:** 将来の装備体系を見据えた先進的な技術",
    "出典": "1ページ「令和７年度予算の概要」内「１．歳出」の表（予算総額と研究開発費）、2ページ「２．主要経費」内「（１）重点的な整備・強化を行う主要分野」の研究開発に関する記述"
  },
  {
    "質問": "令和7年度の防衛予算において、装備品費等は総額の何パーセントを占めていますか？",
    "模範解答": "令和7年度の防衛予算総額は7兆9,170億円であり、装備品費等は2兆8,354億円です。したがって、装備品費等の割合は、\n\n$$\\frac{28,354 \\text{億円}}{79,170 \\text{億円}} \\times 100 \\approx 35.81 \%$$\n",
    "出典": "1ページ「令和７年度予算の概要」内「１．歳出」の表（予算総額と装備品費等）"
  }
]

output_file = datafolder + "Q&A.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data_to_save, f, ensure_ascii=False, indent=2)

print(f"データは '{output_file}' に保存されました。")

データは '/content/drive/MyDrive/Colab Notebooks/AIengineering2025_colab/DAY3/data/Q&A.json' に保存されました。


# 没コード。目次のタイトルでチャンクを分けようとしましたが、検索がうまくいきませんでした。

### **Step3** テキストを整形して目次タイトルごとにチャック

次に整形したテキストのチャンクを目次ページの項目ごとにチャンクをしてみます。

PDF読み込み、整形の部分の関数はこれまでと同じです。

In [ ]:
import re
import json
from PyPDF2 import PdfReader

# ======== 事前定義された整形関数（そのまま貼り付けてください） ========
def normalize_spaces(text): # 無駄な空白を削除する。
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        line = line.strip()

        # 全角文字同士の間の空白を削除
        line = re.sub(r'(?<=[^\x00-\x7F])\s+(?=[^\x00-\x7F])', '', line)

        # 全角 → 半角、半角 → 全角 の間の空白も削除
        line = re.sub(r'(?<=[^\x00-\x7F])\s+(?=[\x00-\x7F])', '', line)  # 全→半
        line = re.sub(r'(?<=[\x00-\x7F])\s+(?=[^\x00-\x7F])', '', line)  # 半→全

        cleaned_lines.append(line)

    return "\n".join(cleaned_lines)

def remove_linebreaks(text):
    # 「。」以外の改行を削除し、文をつなげる
    lines = text.splitlines()
    buffer = ""
    for line in lines:
        line = line.strip()
        if line:
            if line.endswith("。"):
                buffer += line + "\n"
            else:
                buffer += line  # 改行を除去して連結
    return buffer.strip()

def add_missing_linebreaks(text):
    pattern = re.compile(
        r'(?<!\n)'  # 直前が改行でない
        r'('
        r'(?:[○◎※●■◆◇＊①-⑩⑴⑵⑶⑷〇])'  # 全角記号（「・」除外、〇追加）
        r'|(?:\(?\d+[\.\)])'  # (1), 1. など
        r'|(?:第?[０-９\d]+[章部回])'  # 第１章、第２部
        r'|(?:[ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩⅪⅫⅰⅱⅲⅳⅴⅵⅶⅷⅸⅹ])'  # 全角ローマ数字
        r'|(?:\b[IVXLCDM]{1,4}\b)'  # 半角ローマ（大文字）
        r'|(?:\b[ivxlcdm]{1,4}\b)'  # 半角ローマ（小文字）
        r'|(?:(?<![０-９])([０-９]{1,3})(?!年|個|つ|日|円|％|点|名|機|隻|両|式))'  # 全角数字の後に特定の単位がつかない
        # r'|(?:(?<![０-９]))(?![Ａ-Ｚａ-ｚ０-９])'  # 全角英数字（除外）
        r'|(?<![０-９])([○◎※●■◆◇＊①-⑩⑴⑵⑶⑷〇])'  # 全角記号（除外）
        r')'
    )

    def insert_linebreak(match):
        return '\n' + match.group(0)

    return pattern.sub(insert_linebreak, text)

def full_cleaning_pipeline(raw_text):
    text = normalize_spaces(raw_text)
    text = remove_linebreaks(text)
    text = add_missing_linebreaks(text)
    return text

# ======== 整形付きPDFチャンクャークラス ========
class PDFChunkerTocCleend:
    def __init__(self, pdf_path):
        self.reader = PdfReader(pdf_path)
        self.pages = [page.extract_text() for page in self.reader.pages]
        self.pages_text = [full_cleaning_pipeline(page.extract_text() or "") for page in self.reader.pages]

    def find_table_of_contents(self):
      toc_text = ""
      toc_page_idx = -1

      for i, page in enumerate(self.pages):
        if "目次" in page:
          toc_page_idx = i
          toc_text = page
          break
      return toc_page_idx, toc_text.splitlines()

    def insert_linebreaks_before_numbered_items(self, text):
      """
      目次ページ用：行頭以外で改行されていない全角数字項目の直前に改行を追加。
      例：「１防衛２研究」→「１防衛\n２研究」
      """
      # 改行で始まっていない連続数字（全角）に改行を補う（「５防衛力を…６日米…」のようなケースに対応）
      return re.sub(r'(?<!\n)(?=[０-９]{1,2}[^\d０-９\s])', '\n', text)

    def parse_item_list_toc(self, toc_text):
        toc_text = self.insert_linebreaks_before_numbered_items(toc_text)  # ←ここを追加
        lines = toc_text.splitlines()
        toc_titles = []
        for line in lines:
            line = line.strip()
            if 2 < len(line) < 40 and not re.search(r'\d{1,3}$', line):
                toc_titles.append(line)
        return toc_titles


    def chunk_by_titles_in_whole_text(self, titles, toc_index=0):
      full_text = "\n".join(self.pages_text[toc_index+1:])

      print(full_text[:1000])

      def strip_leading_symbols(text):
        # 番号、記号（Ⅰ、Ⅱなど）や余分なスペースを取り除く
        return re.sub(r'^[\s\dⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ十]+[\s．.]*', '', text).strip()

      starts = []

      for title in titles:
        cleaned = strip_leading_symbols(title)  # 目次のタイトルを整形
        match = re.search(re.escape(cleaned), full_text)  # 本文内で一致する位置を検索
        if match:
            starts.append((match.start(), title))  # 一致した位置と元のタイトルを格納
        else:
            print(f"[NOT FOUND] {title} → 検索語: {cleaned}")  # 見つからなかった場合print(starts)

      starts.sort()

      chunks = []
      for i, (start_idx, title) in enumerate(starts):
          end_idx = starts[i+1][0] if i+1 < len(starts) else len(full_text)
          chunk_text = full_text[start_idx:end_idx].strip()
          chunks.append({"title": title, "content": chunk_text})

      print("\nTOC Titles:")
      for title in titles:
          print(f"'{title}'")
      print("\nStart positions found:")
      for start, title in starts:
          print(f"Title: {title}, Start position: {start}")

      return chunks

    # def normalize_for_matching(text):
    #     return re.sub(r'\s|　|－|ー|-|–|―', '', text)

    # def find_title_locations(self, toc_titles):
    #     title_page_map = {}
    #     for title in toc_titles:
    #         norm_title = normalize_for_matching(title)
    #         for i, text in enumerate(self.pages_text):
    #             if norm_title in normalize_for_matching(text) and title not in title_page_map:
    #                 title_page_map[title] = i
    #                 break
    #     return title_page_map

    # def find_title_locations(self, titles):
    #   def normalize(text):
    #       # 数字・記号を取り除いて空白も削除
    #       text = re.sub(r'^[\dⅠⅡⅢIVV]+[\.\s　\-]*', '', text)  # 先頭の番号や記号を削除
    #       return re.sub(r'[\s　\-]', '', text)  # 全ての空白とハイフンを削除

    #   page_map = {}
    #   for title in titles:
    #       norm_title = normalize(title)
    #       found = False
    #       for i, page in enumerate(self.pages_text):
    #           page_text = normalize(page)
    #           if norm_title in page_text:
    #               page_map[title] = i
    #               print(f"[MATCH] '{title}' → Page {i}")
    #               found = True
    #               break
    #       if not found:
    #           print(f"[NOT FOUND] '{title}'")
    #   return page_map

    # def find_title_locations(self, toc_titles):
    #     title_page_map = {}
    #     for title in toc_titles:
    #         title_pattern = re.escape(title).replace(r'\ ', r'\s*')  # スペースを任意の空白に
    #         for i, text in enumerate(self.pages_text):
    #             if title in text and title not in title_page_map:
    #                 title_page_map[title] = i
    #                 break
    #     return title_page_map

    # def chunk_by_titles(self, title_page_map):
    #     sorted_titles = sorted(title_page_map.items(), key=lambda x: x[1])
    #     chunks = []
    #     for i, (title, start) in enumerate(sorted_titles):
    #         end = sorted_titles[i+1][1] if i+1 < len(sorted_titles) else len(self.pages_text)
    #         content = "\n".join(self.pages_text[start:end])
    #         chunks.append((title, content.strip()))
    #     return chunks

    def chunk_by_slide_titles(self):
        chunks = []
        current_title = "Untitled"
        current_content = ""

        for page_text in self.pages_text:
            lines = page_text.splitlines()
            for i, line in enumerate(lines):
                stripped = line.strip()
                if self.is_slide_title(stripped):
                    if current_content:
                        chunks.append((current_title, current_content.strip()))
                    current_title = stripped
                    current_content = "\n".join(lines[i+1:])
                    break
            else:
                current_content += "\n" + page_text

        if current_content:
            chunks.append((current_title, current_content.strip()))
        return chunks

    def is_slide_title(self, line):
        return bool(re.match(r'^(第?[０-９\d]+[章部回]|[①-⑩]|[■◆●〇※・]|[A-Z])', line)) \
            or (len(line) < 25 and not line.endswith(("。", "、", "）")))

    def run(self):
      _, toc_lines = self.find_table_of_contents()
      if toc_lines:
        toc_text = "\n".join(toc_lines)
        toc_titles = self.parse_item_list_toc(toc_text)
        return self.chunk_by_titles_in_whole_text(toc_titles)
      else:
          return self.chunk_by_slide_titles()

    def save_to_json(self, chunks, out_path):
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump([{"title": title, "content": content} for title, content in chunks], f, ensure_ascii=False, indent=2)

In [ ]:
pdf_path = pdfpath
# out_path = datafolder + "pattern3_cleand_TOC.json"

processor = PDFChunkerTOCcleend(pdf_path)
# cleaned = processor.full_cleaning_pipeline()
list_toc = processor.find_table_of_contents()
print(list_toc)

(1, ['目次', 'Ⅰ', '防衛力整備計画の進捗状況', 'Ⅱ', '令和７年度予算', 'Ⅲ', '主要事項', '１スタンド・オフ防衛能力', '２統合防空ミサイル防衛能力', '３無人アセット防衛能力', '４領域横断作戦能力', '５指揮統制・情報関連機能', '６機動展開能力・国民保護', '７持続性・強靱性', 'Ⅳ', '共通基盤', '１防衛生産基盤の強化', '２研究開発', '３早期装備化のための取組', '４ＡＩ活用の推進に係る施策', '５防衛力を支える要素', '６日米同盟強化及び地域社会との調和に係る施策等', '７安全保障協力の強化', '８気候変動への取組', '９最適化への取組', '１０ 自衛隊の組織編成', '１１ 自衛官の定員', '１２ 事務官等の増員', '１３ 情報保全の強化', '１４ 監察体制の強化', '１５ 税制改正1'])


In [ ]:
pdf_path = pdfpath
# out_path = datafolder + "pattern3_cleand_TOC.json"

processor = PDFChunkerTocCleend(pdf_path)
# cleaned = processor.full_cleaning_pipeline()
tocchunks = processor.run()
processor.save_to_json(tocchunks, out_path)

目次
Ⅰ防衛力整備計画の進捗状況
Ⅱ令和７年度予算
Ⅲ主要事項
１スタンド・オフ防衛能力
２統合防空ミサイル防衛能力
３無人アセット防衛能力
４領域横断作戦能力
５指揮統制・情報関連機能６機動展開能力・国民保護
７持続性・強靱性
Ⅳ共通基盤
１防衛生産基盤の強化
２研究開発
３早期装備化のための取組
４ＡＩ活用の推進に係る施策
５防衛力を支える要素６日米同盟強化及び地域社会との調和に係る施策等
７安全保障協力の強化
８気候変動への取組
９最適化への取組
１０自衛隊の組織編成
１１自衛官の定員
１２事務官等の増員
１３情報保全の強化
１４監察体制の強化
１５税制改正1
7つの重視分野抜本的強化の進捗状況(一例)スタンド・オフ防衛能力
〇トマホークの取得１年前倒し(2026年度→2025年度)
〇トマホーク運用のための海上自衛官等に対する要員養成教育を開始(2024年3月～10月)
〇島嶼防衛用高速滑空弾の事前発射試験実施(2024年4月)
〇
１２式地対艦誘導弾能力向上型(地発型)の配備１年前倒し(2026年度→2025年度)
〇12式地対艦誘導弾能力向上型の発射試験実施(2024年10～11月)統合防空ミサイル防衛能力
○イージス・システム搭載艦2隻の建造契約を締結(2024年8・9月)
○ＧＰＩの日米共同開発に係る開発コンセプトを決定(2024年9月)
※GPI: Glide Phase Interceptor(滑空段階迎撃用誘導弾)無人アセット防衛能力
○2023年度中に合計
１０機種の各種ＵＡＶの運用実証を実施
○各種無人アセットに関する研究開発を推進
○滞空型ＵＡＶの機種として「ＭＱ―
９Ｂ(シーガーディアン)」を選定(2024年11月)
Ⅰ防衛力整備計画の進捗状況
○我が国の防衛上必要な機能・能力として、次の７つの分野を重視して、防衛力の抜本的強化を推進。
○事業の進捗管理を徹底し、概ね計画どおり進捗。
○令和７年度予算においても、計画期間内の防衛力抜本的強化実現のため、令和７年度中に着手すべき事業を計上。
島嶼防衛用高速滑空弾トマホーク(イメージ)イージス・システム搭載艦(イメージ)【研究開発】長期運用型UUV(イメージ)2
１２式地対艦誘導弾能力向上型滞空型UAV
7つの重視分野抜本的強化の進捗状況(一例)領域横断作戦能力(宇宙・サイバー・陸海空領域)

In [ ]:
processor.full_text[:1000]

AttributeError: 'PDFChunkerTocCleend' object has no attribute 'full_text'

In [ ]:
processor.pages_text

['防衛省Ministry of  Defense防衛力抜本的強化の進捗と予算令和７年度予算の概要',
 '目次\nⅠ防衛力整備計画の進捗状況\nⅡ令和７年度予算\nⅢ主要事項\n１スタンド・オフ防衛能力\n２統合防空ミサイル防衛能力\n３無人アセット防衛能力\n４領域横断作戦能力\n５指揮統制・情報関連機能６機動展開能力・国民保護\n７持続性・強靱性\nⅣ共通基盤\n１防衛生産基盤の強化\n２研究開発\n３早期装備化のための取組\n４ＡＩ活用の推進に係る施策\n５防衛力を支える要素６日米同盟強化及び地域社会との調和に係る施策等\n７安全保障協力の強化\n８気候変動への取組\n９最適化への取組\n１０自衛隊の組織編成\n１１自衛官の定員\n１２事務官等の増員\n１３情報保全の強化\n１４監察体制の強化\n１５税制改正1',
 '7つの重視分野抜本的強化の進捗状況(一例)スタンド・オフ防衛能力\n〇トマホークの取得１年前倒し(2026年度→2025年度)\n〇トマホーク運用のための海上自衛官等に対する要員養成教育を開始(2024年3月～10月)\n〇島嶼防衛用高速滑空弾の事前発射試験実施(2024年4月)\n〇\n１２式地対艦誘導弾能力向上型(地発型)の配備１年前倒し(2026年度→2025年度)\n〇12式地対艦誘導弾能力向上型の発射試験実施(2024年10～11月)統合防空ミサイル防衛能力\n○イージス・システム搭載艦2隻の建造契約を締結(2024年8・9月)\n○ＧＰＩの日米共同開発に係る開発コンセプトを決定(2024年9月)\n※GPI: Glide Phase Interceptor(滑空段階迎撃用誘導弾)無人アセット防衛能力\n○2023年度中に合計\n１０機種の各種ＵＡＶの運用実証を実施\n○各種無人アセットに関する研究開発を推進\n○滞空型ＵＡＶの機種として「ＭＱ―\n９Ｂ(シーガーディアン)」を選定(2024年11月)\nⅠ防衛力整備計画の進捗状況\n○我が国の防衛上必要な機能・能力として、次の７つの分野を重視して、防衛力の抜本的強化を推進。\n○事業の進捗管理を徹底し、概ね計画どおり進捗。\n○令和７年度予算においても、計画期間内の防衛力抜本的強化実現のため、令和７年度中に着手すべき事業を計上。\n島嶼防衛用高速滑空弾トマホーク(イメージ)イージス

In [ ]:
import json

with open(out_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 最初の3件だけ表示
for i, item in enumerate(data[:10]):
    print(f"--- Chunk {i+1} ---")
    print("Title:", item["title"])
    print("Content:", item["content"][:2000], "...\n")  # 長すぎる場合は先頭だけ

--- Chunk 1 ---
Title: title
Content: content ...

--- Chunk 2 ---
Title: title
Content: content ...

--- Chunk 3 ---
Title: title
Content: content ...

--- Chunk 4 ---
Title: title
Content: content ...

--- Chunk 5 ---
Title: title
Content: content ...

--- Chunk 6 ---
Title: title
Content: content ...

--- Chunk 7 ---
Title: title
Content: content ...

--- Chunk 8 ---
Title: title
Content: content ...

--- Chunk 9 ---
Title: title
Content: content ...

--- Chunk 10 ---
Title: title
Content: content ...

